# Parkway Project Use Case 1: Write Off Cases Prediction

## DATA PREPARATION

### Load Data

#### Import packages

In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
import os
os.chdir('d:/Users/mokky/Documents/GitHub/nus-iss/PRS-PM-ISY5002-GROUP5/SystemCode')
print(os.getcwd())

d:\Users\mokky\Documents\GitHub\nus-iss\PRS-PM-ISY5002-GROUP5\SystemCode


#### Read data from file

In [3]:
dict_X_train_file_paths = {
    'GHL' : ['./Data Exploration/data/GHL_data_X_train_0.pkl',
             './Data Exploration/data/GHL_data_X_train_1.pkl',
             './Data Exploration/data/GHL_data_X_train_2.pkl',
             './Data Exploration/data/GHL_data_X_train_3.pkl',
             './Data Exploration/data/GHL_data_X_train_4.pkl',
             './Data Exploration/data/GHL_data_X_train_5.pkl',
             './Data Exploration/data/GHL_data_X_train_6.pkl',
             './Data Exploration/data/GHL_data_X_train_7.pkl'],
    'MEH' : ['./Data Exploration/data/MEH_data_X_train_0.pkl',
             './Data Exploration/data/MEH_data_X_train_1.pkl',
             './Data Exploration/data/MEH_data_X_train_2.pkl',
             './Data Exploration/data/MEH_data_X_train_3.pkl',
             './Data Exploration/data/MEH_data_X_train_4.pkl',
             './Data Exploration/data/MEH_data_X_train_5.pkl',
             './Data Exploration/data/MEH_data_X_train_6.pkl',
             './Data Exploration/data/MEH_data_X_train_7.pkl',
             './Data Exploration/data/MEH_data_X_train_8.pkl',
             './Data Exploration/data/MEH_data_X_train_9.pkl'],
    'PEH' : ['./Data Exploration/data/PEH_data_X_train_0.pkl',
             './Data Exploration/data/PEH_data_X_train_1.pkl',
             './Data Exploration/data/PEH_data_X_train_2.pkl'],
    'PNH' : ['./Data Exploration/data/PNH_data_X_train_0.pkl',
             './Data Exploration/data/PNH_data_X_train_1.pkl',
             './Data Exploration/data/PNH_data_X_train_2.pkl',
             './Data Exploration/data/PNH_data_X_train_3.pkl',
             './Data Exploration/data/PNH_data_X_train_4.pkl',
             './Data Exploration/data/PNH_data_X_train_5.pkl',
             './Data Exploration/data/PNH_data_X_train_6.pkl',
             './Data Exploration/data/PNH_data_X_train_7.pkl']}
dict_y_train_file_paths = {
    'GHL' : './Data Exploration/data/GHL_data_y_train.pkl',
    'MEH' : './Data Exploration/data/MEH_data_y_train.pkl',
    'PEH' : './Data Exploration/data/PEH_data_y_train.pkl',
    'PNH' : './Data Exploration/data/PNH_data_y_train.pkl'}
dict_X_test_file_paths = {
    'GHL' : ['./Data Exploration/data/GHL_data_X_test_0.pkl',
             './Data Exploration/data/GHL_data_X_test_1.pkl',
             './Data Exploration/data/GHL_data_X_test_2.pkl'],
    'MEH' : ['./Data Exploration/data/MEH_data_X_test_0.pkl',
             './Data Exploration/data/MEH_data_X_test_1.pkl',
             './Data Exploration/data/MEH_data_X_test_2.pkl',
             './Data Exploration/data/MEH_data_X_test_3.pkl'],
    'PEH' : ['./Data Exploration/data/PEH_data_X_test_0.pkl'],
    'PNH' : ['./Data Exploration/data/PNH_data_X_test_0.pkl',
             './Data Exploration/data/PNH_data_X_test_1.pkl',
             './Data Exploration/data/PNH_data_X_test_2.pkl']}
dict_y_test_file_paths = {
    'GHL' : './Data Exploration/data/GHL_data_y_test.pkl',
    'MEH' : './Data Exploration/data/MEH_data_y_test.pkl',
    'PEH' : './Data Exploration/data/PEH_data_y_test.pkl',
    'PNH' : './Data Exploration/data/PNH_data_y_test.pkl'}

In [4]:
for hosp in dict_X_train_file_paths:
    df_X_train = pd.concat([pd.read_pickle(file_path)
                                  for file_path in dict_X_train_file_paths[hosp]])
    print(f'X_train {hosp} {df_X_train.shape}')
    
for hosp in dict_y_train_file_paths:
    df_y_train = pd.read_pickle(dict_y_train_file_paths[hosp])
    print(f'y_train {hosp} {df_y_train.shape}')

for hosp in dict_X_test_file_paths:
    df_X_test = pd.concat([pd.read_pickle(file_path)
                                  for file_path in dict_X_test_file_paths[hosp]])
    print(f'X_test {hosp} {df_X_test.shape}')

for hosp in dict_y_test_file_paths:
    df_y_test = pd.read_pickle(dict_y_test_file_paths[hosp])
    print(f'y_test {hosp} {df_y_test.shape}')

X_train GHL (79672, 38885)
X_train MEH (98296, 38885)
X_train PEH (28604, 38885)
X_train PNH (79336, 38885)
y_train GHL (79672,)
y_train MEH (98296,)
y_train PEH (28604,)
y_train PNH (79336,)
X_test GHL (26557, 38885)
X_test MEH (32766, 38885)
X_test PEH (9534, 38885)
X_test PNH (26445, 38885)
y_test GHL (26557,)
y_test MEH (32766,)
y_test PEH (9534,)
y_test PNH (26445,)


## Get feature Importances

In [7]:
from sklearn.ensemble import RandomForestRegressor

impt_threshold = 0.0

dict_forest_file_paths = {}
dict_feat_impt_file_paths = {}
dict_new_X_train_file_paths = {}
dict_new_X_test_file_paths = {}
for hosp in dict_X_train_file_paths:
    save_folder_paths = os.path.dirname(dict_X_train_file_paths[hosp][0])
    
    df_X_train = pd.concat([pd.read_pickle(file_path)
                            for file_path in dict_X_train_file_paths[hosp]])
    print(f'X_train {hosp} {df_X_train.shape}')
    
    df_y_train = pd.read_pickle(dict_y_train_file_paths[hosp])
    print(f'y_train {hosp} {df_y_train.shape}')
    
    # forest
    forest = RandomForestRegressor(random_state=42)
    forest.fit(df_X_train, df_y_train)
    
    forest_pkl_file_path = f'{save_folder_paths}/{hosp}_forest_model_uc1.pkl'
    with open(forest_pkl_file_path, 'wb') as file:
        pickle.dump(forest, file)
    dict_forest_file_paths[hosp] = forest_pkl_file_path
    
    # feature importance
    print(f'{sum(forest.feature_importances_ > impt_threshold)} / {len(forest.feature_importances_)}')
    feature_importance_file_path = f'{save_folder_paths}/{hosp}_forest_feat_impt_uc1.npy'
    np.save(feature_importance_file_path, forest.feature_importances_ > impt_threshold)
    dict_feat_impt_file_paths[hosp] = feature_importance_file_path
    
    # new training files
    df_train_new_X = df_X_train.loc[:, forest.feature_importances_ > impt_threshold]
    print(df_train_new_X.shape)
    new_X_train_pkl_file_path = f'{save_dir}/{hosp}_new_train_X_uc1.pkl'
    df_train_new_X.to_pickle(new_X_train_pkl_file_path)
    dict_new_X_train_file_paths[hosp] = new_X_train_pkl_file_path
    
    # new test files
    df_test_new_X = df_X_test.loc[:, forest.feature_importances_ > impt_threshold]
    print(df_test_new_X.shape)
    new_X_test_pkl_file_path = f'{save_dir}/{hosp}_new_test_X_uc1.pkl'
    df_test_new_X.to_pickle(new_X_test_pkl_file_path)
    dict_new_X_test_file_paths[hosp] = new_X_test_pkl_file_path

print('forest:\n', dict_forest_file_paths)
print('feature importance:\n', dict_feat_impt_file_paths)
print('New X_train:\n', dict_new_X_train_file_paths)
print('New X_test:\n', dict_new_X_test_file_paths)

X_train GHL (79672, 38885)
y_train GHL (79672,)


KeyboardInterrupt: 